In [1]:
from keras import layers, models, regularizers, backend
from keras.layers import (
    Activation,
    BatchNormalization,
    Conv1D,
    Conv2D,
    Conv3D,
    ConvLSTM1D,
    ConvLSTM2D,
)
from keras.layers.core import Lambda
reg_weight = 1.0e-4

In [2]:
def dense_block(x, blocks, name):
    """A dense block.
    Args:
      x: input tensor.
      blocks: integer, the number of building blocks.
      name: string, block label.
    Returns:
      Output tensor for the block.
    """
    for i in range(blocks):
        x = dense_conv_block(x, 32, name=name + "_block" + str(i + 1))
    return x


def transition_block(x, reduction, name):
    """A transition block.
    Args:
      x: input tensor.
      reduction: float, compression rate at transition layers.
      name: string, block label.
    Returns:
      output tensor for the block.
    """
    bn_axis = -1 
    x = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_bn"
    )(x)
    x = layers.Activation("relu", name=name + "_relu")(x)
    x = layers.Conv3D(
        int(backend.int_shape(x)[bn_axis] * reduction),
        1,
        use_bias=False,
        name=name + "_conv",
    )(x)
    x = layers.AveragePooling3D(2, strides=1, name=name + "_pool")(x)
    return x


def dense_conv_block(x, growth_rate, name):
    """A building block for a dense block.
    Args:
      x: input tensor.
      growth_rate: float, growth rate at dense layers.
      name: string, block label.
    Returns:
      Output tensor for the block.
    """
    bn_axis = -1 
    x1 = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_0_bn"
    )(x)
    x1 = layers.Activation("relu", name=name + "_0_relu")(x1)
    x1 = layers.Conv3D(
        4 * growth_rate, 1, use_bias=False, name=name + "_1_conv"
    )(x1)
    x1 = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_1_bn"
    )(x1)
    x1 = layers.Activation("relu", name=name + "_1_relu")(x1)
    x1 = layers.Conv3D(
        growth_rate, 3, padding="same", use_bias=False, name=name + "_2_conv"
    )(x1)
    x = layers.Concatenate(axis=bn_axis, name=name + "_concat")([x, x1])
    return x



def densenet_3D_layer(
    inputs,
    num_filters = 64,
    kernel_size = 3,
    strides = 1,
    activation='relu',
   
):
    
    x = inputs
    x = layers.Conv3D(
        num_filters,
        kernel_size=kernel_size,
        strides=strides,
        padding="same",
        kernel_initializer="he_normal",
        kernel_regularizer=regularizers.l2(1e-4),
    )(inputs)
    x = layers.BatchNormalization(
        axis= -1, epsilon=1.001e-5, name="conv1/bn"
    )(x)
    x = layers.Activation(activation, name="conv1activation")(x)
    
    return x


In [3]:
def DenseVollNet(
    input_shape,
    categories,
    box_vector,
    nboxes=1,
    blocks=[6, 12, 24, 16],
    start_kernel=7,
    mid_kernel=3,
    startfilter=32,
    input_weights=None,
    last_activation="softmax",
):
    print(input_shape, input_shape[0], input_shape[1], input_shape[2], input_shape[3])
    img_input = layers.Input(shape=(input_shape[0], input_shape[1], input_shape[2], input_shape[3]))
    bn_axis = -1
    num_filters_in = startfilter
    x = densenet_3D_layer( 
        inputs=img_input,
        num_filters=num_filters_in,
        kernel_size=start_kernel,
        strides = 1)
    # Start model definition.
    
    x = dense_block(x, blocks[0], name="conv2")
    x = transition_block(x, 0.5, name="pool2")
    
    x = dense_block(x, blocks[1], name="conv3")
    x = transition_block(x, 0.5, name="pool3")
    
    x = dense_block(x, blocks[2], name="conv4")
    x = transition_block(x, 0.5, name="pool4")
    
    
    
    x = dense_block(x, blocks[3], name="conv5")

    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name="bn")(x)
    x = layers.Activation("relu", name="relu")(x)
    x = (
        Conv3D(
            categories + nboxes * box_vector,
            kernel_size=mid_kernel,
            kernel_regularizer=regularizers.l2(reg_weight),
            padding="same",
        )
    )(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    inputs = img_input
    outputs = x
    # Create model.
    model = models.Model(inputs, outputs)

    if input_weights is not None:

        model.load_weights(input_weights, by_name=True)

    return model

In [4]:
model = DenseVollNet(
    (8,64,64,3),
    2,
    8,
    nboxes=1,
    blocks=[6, 12, 24, 16],
    start_kernel=7,
    mid_kernel=3,
    startfilter=32,
    input_weights=None,
    last_activation="softmax",
)
model.summary()

(8, 64, 64, 3) 8 64 64 3
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8, 64, 64,   0           []                               
                                3)]                                                               
                                                                                                  
 conv3d (Conv3D)                (None, 8, 64, 64, 3  32960       ['input_1[0][0]']                
                                2)                                                                
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 8, 64, 64, 3  128         ['conv3d[0][0]']                 
                                2)                                   